In [ ]:
import pickle
with open("m3_100_nn_test_traj_pred_list.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
import pickle as pickle
import numpy as np
base_path="../data_m3/"

with open(base_path+'binary_train_codes_x.pkl', 'rb') as f0:
  binary_train_codes_x = pickle.load(f0)

with open(base_path+'binary_test_codes_x.pkl', 'rb') as f1:
  binary_test_codes_x = pickle.load(f1)

train_codes_y = np.load(base_path+'train_codes_y.npy')
train_visit_lens = np.load(base_path+'train_visit_lens.npy')
test_codes_y = np.load(base_path+'test_codes_y.npy')
test_visit_lens = np.load(base_path+'test_visit_lens.npy')
train_pids = np.load(base_path+'train_pids.npy')
test_pids = np.load(base_path+'test_pids.npy')
with open(base_path+'patient_time_duration_encoded.pkl', 'rb') as f80:
  patient_time_duration_encoded = pickle.load(f80)

In [ ]:
pred=[]
for i in range(len(data)):
    pred.append(data[i][-1])
pred=np.vstack(pred)
pred = np.argsort(-pred, axis=-1)
y_prob=pred

y_true=[]
for i in range(len(test_codes_y)):
    if test_visit_lens[i]<2:
        continue
    y_true.append(test_codes_y[i])
y_true=np.vstack(y_true)

In [ ]:
import numpy as np
import pickle

with open(base_path+"code_map.pkl", "rb") as f:
    code_map = pickle.load(f)

inv_code_map = {v-1: k for k, v in code_map.items()}
print(code_map)
def fun1(idx):
    result=[]
    for id in idx:
        result.append(inv_code_map[id])
    return result

In [ ]:
import pandas as pd
df = pd.read_csv(base_path+'diagnosis_icd9_ontology.csv')
code2name = dict(zip(df['code'], df['name']))

In [ ]:
def get_diag_str(diag_list):
    temp_str_list=[]
    for diag in diag_list:
        temp_str_list.append(f'"{code2name[diag]}"')
    temp_str=', '.join(temp_str_list)
    return temp_str

In [ ]:
import inflect
def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()

In [ ]:
ode_idx=0
results=[]
top_k=90

for i in range(len(test_codes_y)):
    if test_visit_lens[i]<2:
        continue
    pids=test_pids[i]
    ode_pred=data[ode_idx]
    ode_pred = np.argsort(-ode_pred, axis=-1)
    summary=""
    hist_diag=binary_test_codes_x[i]
    target_diag=test_codes_y[i]
    ode_idx+=1
    seq_len=test_visit_lens[i]+1
    hist_diag_code=[]
    for v in hist_diag:
        hist_diag_code.append(fun1(np.where(v == 1)[0]))

    ode_pred_code=[]
    for v_idx, v in enumerate(ode_pred[:len(hist_diag_code)]):
        gt = hist_diag_code[v_idx]
        gt_len = len(gt)
        ode_pred_code.append(list(fun1(v[:gt_len])))

    missed_diseases_all=[]
    missed_diseases_all_set = []
    for v_idx in range(len(hist_diag_code)):
        gt = hist_diag_code[v_idx]
        pred = set(ode_pred_code[v_idx])
        missed = [d for d in gt if d not in pred]
        missed_diseases_all_set.extend(missed)
        missed_diseases_all.append(missed)

    wrong_diseases_all = []
    wrong_diseases_all_set = []
    for v_idx in range(len(ode_pred_code)):
        pred = ode_pred_code[v_idx]
        gt = set(hist_diag_code[v_idx])
        wrong = [d for d in pred if d not in gt]
        wrong_diseases_all_set.extend(wrong)
        wrong_diseases_all.append(wrong)
    
    wrong_diseases_all_set=list(set(wrong_diseases_all_set))
    missed_diseases_all_set=list(set(missed_diseases_all_set))
    duration=patient_time_duration_encoded[pids]
    instruction_summary=("You are an experienced medical diagnosis expert who understands disease progression, comorbidity patterns, and model prediction bias.\n"
                +"Your task involves two objectives:\n"
                +"1.Carefully analyze the patient's true disease trajectory across multiple visits and summarize key patterns, including major chronic conditions, emerging complications, and notable progression trends over time.\n"
                +"2.Analyze the model's prediction errors (missed or wrongly predicted diseases) and provide possible reasons for these errors, such as atypical disease progression, rare co-occurrences, or data sparsity.\n"
                +"You can use the following to guide your thinking:\n"
                +"- The patient’s true (ground-truth) diagnoses per visit.\n"
                +"- Diagnoses that the model missed and diagnoses it predicted incorrectly.\n"
                +"- Your medical knowledge and understanding of typical disease co-occurrence and evolution.\n")
    input_summary=""
    for idx,code_list in enumerate(hist_diag_code):
        if idx==0:
            cur_duration=0
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit:"
        else:
            cur_duration+=int(duration[idx])
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit: ({cur_duration} days later):"
        temp+=f"\n- Ground-Truth Diagnoses："+"{"+get_diag_str(code_list)+"}"
        temp+=f"\n- True diagnoses that were missed by the model："+"{"+get_diag_str(missed_diseases_all[idx])+"}"
        temp+=f"\n- Incorrect diagnoses predicted by the model："+"{"+get_diag_str(wrong_diseases_all[idx])+"}"
        input_summary += temp+"\n\n"

    temp =( "Your task:\n"
           +"- Summarize the patient's overall disease characteristics and progression pattern across all visits, highlighting the major chronic conditions, emerging complications, and temporal evolution of the diseases.\n"
           +"- Analyze the model's prediction errors for this patient, including missed diagnoses and incorrectly predicted diagnoses, and provide possible reasons for these errors based on disease progression patterns, comorbidity relationships, or data sparsity.\n"
           +"- Focus on capturing how the patient's conditions have developed, worsened, or interacted over time, and on clinically plausible explanations for model mistakes.\n\n"
           +"Output format:\n"
           +"Summary: <a single sentence describing the patient's main disease features and progression pattern>\n"
           +"Error Analysis: <one concise paragraph explaining likely causes of the model's prediction errors>")
    input_summary+=temp+"\n"
    instruction=("You are an experienced medical diagnosis expert who understands disease progression, comorbidity patterns.\n"
                 +f"Your goal is to re-rank the model’s top-{top_k} predicted diseases for the final visit of a patient.\n"+f"You can use the following to guide your re-ranking:\n- Summary: <A single sentence summarizing the patient’s main disease features and progression pattern.>\n- Error Analysis: <One concise paragraph describing the likely causes of the model’s prediction errors.>\n- Incorrect Diagnoses Predicted by the Model: <List of diseases that were incorrectly predicted.>\n- True Diagnoses Missed by the Model: <List of ground-truth diseases that the model failed to predict.>\n- The patient’s Ground-Truth Diagnoses per visit.")
    input_="\n"
    input_+=summary
    input_+="\n\nIncorrect diagnoses predicted by the model: \n"+get_diag_str(wrong_diseases_all_set)+"\n"
    input_+="\nTrue diagnoses that were missed by the model: \n"+get_diag_str(missed_diseases_all_set)+"\n"
    for idx,code_list in enumerate(hist_diag_code):
        if idx==0:
            cur_duration=0
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit:"
        else:
            cur_duration+=int(duration[idx])
            temp=number_to_capitalized_ordinal(idx+1)+f" Visit: ({cur_duration} days later):"
        temp+=f"\n- Ground-Truth Diagnoses："+"{"+get_diag_str(code_list)+"}"
        input_+=temp+"\n\n"
    temp=f"Final Visit ({int(sum(duration))} days later):"
    temp+=f"\n- Model’s Top-{top_k} Predicted Diseases: "+"{"+get_diag_str(list(fun1(ode_pred[-1][:top_k])))+"}\n"
    temp+=f"\n\nYour task:\n- Re-rank the final visit’s candidate diseases in descending order of likelihood, based on your learned correction and medical reasoning."+"\nDirectly provide the reordered list of disease names in descending order of likelihood. \nOutput format:\nAnswer: <Disease 1>, <Disease 2>, <Disease 3>, ..."
    input_+=temp+"\n"
    result={
        "pid":pids,
        "seq_len":seq_len,
        "instruction_summary":instruction_summary,
        "input_summary":input_summary,
        "instruction":instruction,
        "input":input_
    }
    results.append(result)

In [ ]:
import json
def np_encoder(obj):
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return str(obj)

with open(f"../prompt_data/TempCare_true_top50.json", "w",encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False,default=np_encoder, indent=2)